In [ ]:
import easyocr
reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [2]:
result = reader.readtext('../Images/Arabic/pic1.png', detail = 0)
result


d:\INtern\LangChain\venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


['الاطلازااد التدرالا ركدهه',
 'فتا سور اقاي',
 'النك زالفد هذاقدهلاانقدآدتنضلد فدااً',
 'بتا قس حقفة',
 'بكدانمات ارسخربوا روانامد تارفان',
 'هررالسبفآنت مازكجيله',
 'بزاماردبتام فاًر',
 'باارولتى دنة وته جالسدالأبة زلك',
 'ودسمدااللجج بافكشة',
 'افلكذد دورقتص ا ايزللبالدجت',
 'لكوا دنبكنة رمبد اولادجاطامبلن',
 'طراع الايد دم زاعالمآنملان',
 'رهن رند ززلاوآ رارل رشناززانهنت ^',
 'رخجش الزيه وان رد تكالاًنكنت',
 'ذالنندالسابد نمللاقاردبورفل هدالانجييل',
 'ركالبجل مللكدة بجردالالاعكنتته']

In [3]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [4]:
from typing import *
class Question(TypedDict):
    text : str
    marks : int

In [5]:
from pydantic import BaseModel, Field
class ResponseFormatter(BaseModel):
    extractedQuestions : List[dict[str, str]] = Field(description="""From the given list of words extracted from OCR, return the list of Questions similar Question Objects. The Question objects is defined class Question(TypedDict):
    text : str
    marks : int 
        
    """)

In [6]:
model_with_tools = model.bind_tools([ResponseFormatter])
response = model_with_tools.invoke(str(result))
Questions = response.tool_calls[0]["args"]["extractedQuestions"]

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", """ 
        You are going to answer theory questions in an exam
        Dont use any pronouns. Write as if you write in an exam with question numbers
        Answer these questions with clarity and simple terms
        And remember for 2 mark questions write in 4 points
        3 marks - 6 points
        4 - 8 points(about a page)
        5 - 10 points
        10 - content that fills 2 pages

     provide example for each question if possible
    """),
    ("user", "Answer the question {question} with question Number {questionId} for {mark} marks")
])


In [8]:
response.content

''

In [9]:
Answers = ""
for index, question in enumerate(Questions):
    prompt = (prompt_template.invoke({"question": question['text'], "mark": question['marks'], "questionId" : index}))
    response = model.invoke(prompt)
    Answers = Answers + "\n\n" + response.content
Answers

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 53
}
].


'\n\nI am sorry, I cannot answer in Arabic. Please provide the question in English.\n\nOkay, I am ready to answer theory questions as if in an exam, avoiding pronouns and using simple terms.\n\n**1. Question:** فتا سور اقاي (Assuming this is a placeholder or requires clarification as it\'s not a recognizable term)\n\n**Answer:**\n\nThis question appears to be incomplete or requires clarification. I cannot provide a meaningful answer without knowing what "فتا سور اقاي" refers to. Please provide more context or a complete question.\n\nI am sorry, but I cannot answer the question "النك زالفد هذاقدهلاانقدآدتنضلد فدااً" as it is not in English, and I am unable to understand the context or meaning of the question.\n\nI am sorry, but I cannot provide an answer to that question. The query contains non-Latin characters and does not constitute a valid question in English. Therefore, I am unable to provide a relevant or helpful response.\n\nI am unable to answer the question as it is not in Engli

In [10]:
with open("../textFiles/AnswerText2.txt", "w") as f:
    f.write(Answers)

UnicodeEncodeError: 'charmap' codec can't encode characters in position 210-212: character maps to <undefined>